In [1]:
import pandas as pd
import numpy as np

In [2]:
merged = pd.read_csv('merge.csv')

In [3]:
merged = merged.drop(['application_id', 'bank_id', 'product_id', 'loanapply_insert_time', 'user_id', 'birth_year', 'gender', 'insert_time'],
                       axis = 1)

In [4]:
merged.loc[merged['loan_limit'] != merged['loan_limit'], 'loan_limit'] = merged['loan_limit'].mean()
merged.loc[merged['loan_rate'] != merged['loan_rate'], 'loan_rate'] = merged['loan_rate'].mean()
merged.loc[merged['personal_rehabilitation_yn'] != merged['personal_rehabilitation_yn'], 'personal_rehabilitation_yn'] = 0
merged.loc[merged['personal_rehabilitation_complete_yn'] != merged['personal_rehabilitation_complete_yn'], 'personal_rehabilitation_complete_yn'] = 0
merged = merged.dropna(subset=['credit_score', 'existing_loan_cnt', 'working_period'], axis=0)

In [5]:
merged['tmp'] = merged['existing_loan_amt'] / merged['existing_loan_cnt']
tmp_mean = merged['tmp'].mean()

In [6]:
merged.loc[merged['existing_loan_amt'] != merged['existing_loan_amt'], 'existing_loan_amt'] = tmp_mean

In [7]:
merged = merged.drop(['tmp'], axis=1)
training_set = merged.dropna(subset=['is_applied'], axis=0)

In [8]:
x = training_set.loc[:, training_set.columns != 'is_applied']
y = training_set.loc[:, training_set.columns == 'is_applied']

for col in x:
    if col == 'income_type' or col == 'employment_type' or col == 'houseown_type' or col == 'purpose':
        x[col] = x[col].astype('str')
    elif col == 'loan_rate':
        x[col] = x[col].astype('float')
    else:
        x[col] = x[col].astype('int')

/var/folders/16/4vbkj73s3qlb5w1xc7nxm92m0000gn/T/ipykernel_34639/3256280560.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[col] = x[col].astype('int')
/var/folders/16/4vbkj73s3qlb5w1xc7nxm92m0000gn/T/ipykernel_34639/3256280560.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[col] = x[col].astype('float')
/var/folders/16/4vbkj73s3qlb5w1xc7nxm92m0000gn/T/ipykernel_34639/3256280560.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [9]:
x_dic = x.to_dict('records')

In [10]:
from sklearn.feature_extraction import DictVectorizer
one_hot_encoder = DictVectorizer()
onehot_X = one_hot_encoder.fit_transform(x_dic)

In [11]:
print(onehot_X.shape)
print(y.shape)

(7607193, 32)
(7607193, 1)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

train_X, test_X = onehot_X[0 : 5000000], onehot_X[5000000:]
train_Y, test_Y = y[0 : 5000000], y[5000000:]

decision_tree = DecisionTreeClassifier(criterion = 'gini', min_samples_split=30)
parameters = {'max_depth': [3, 10, None]}

grid_search = GridSearchCV(decision_tree, parameters, cv=3, scoring='accuracy')

grid_search.fit(train_X, train_Y)
print(grid_search.best_params_)

In [ ]:
from sklearn.metrics import classification_report
model_best = grid_search.best_estimator_
prediction = model_best.predict(test_X)

print(classification_report(test_Y, prediction))